# KING

In [20]:
from __future__ import division
from pandas import concat, read_csv, to_datetime
from ggplot import *
from sklearn import linear_model
import pandas as pd
import numpy as np
from numpy import floor, histogram
from scipy import stats
from scipy.stats import gaussian_kde
from sklearn import linear_model, svm

teams = read_csv('./data/2015/team.csv', index_col=0)
statistics = read_csv('./data/2015/team-game-statistics.csv', index_col=0)
games = teams.join(statistics)

for column in games:
    if 'Unnamed' in column:
        del games[column]

games.Date = to_datetime(games.Date, format='%Y%m%d')

winning_scores = games.groupby('Game Code')['Points'].max()
losing_scores = games.groupby('Game Code')['Points'].min()
scores = pd.DataFrame(data={'Losing Points': losing_scores.values, 'Winning Points': winning_scores.values}, index=winning_scores.index)
games = games.join(scores, on='Game Code')

def is_win(game):
    return game['Points'] > game['Losing Points']

def margin(game):
    if is_win(game):
        return game['Points'] - game['Losing Points']
    else:
        return game['Points'] - game['Winning Points']

games['Margin'] = games.apply(margin, axis=1)
games['Is Win'] = games.apply(is_win, axis=1)

def range_bin(array, step):
    return range(int(array.min() / step) * step, int(array.max() / step) * step + step, step)

def histogram_random(data, bins, samples):
    hist, bins = np.histogram(data, bins=bins*2)

    bin_midpoints = bins[:-1] + np.diff(bins)/2
    cdf = np.cumsum(hist)
    cdf = cdf / cdf[-1]
    values = np.random.rand(samples)
    value_bins = np.searchsorted(cdf, values)
    return bin_midpoints[value_bins]

def kde_random(data, samples):
    def kde(x, x_grid):
        kde = gaussian_kde(x)
        return kde.evaluate(x_grid)
    try:
        x_grid = np.linspace(min(data), max(data), samples)
        pdf = kde(data, x_grid)
        cdf = np.cumsum(pdf)
        cdf = cdf / cdf[-1]
        values = np.random.rand(samples)
        value_bins = np.searchsorted(cdf, values)
        random_from_cdf = x_grid[value_bins]
        return random_from_cdf
    except:
        return [0] * samples

feature_columns = ['Rush Att', 'Rush Yard',
       'Rush TD', 'Pass Att', 'Pass Comp', 'Pass Yard', 'Pass TD',
       'Pass Int', 'Pass Conv', 'Kickoff Ret', 'Kickoff Ret Yard',
       'Kickoff Ret TD', 'Punt Ret', 'Punt Ret Yard', 'Punt Ret TD',
       'Fum Ret', 'Fum Ret Yard', 'Fum Ret TD', 'Int Ret', 'Int Ret Yard',
       'Int Ret TD', 'Misc Ret', 'Misc Ret Yard', 'Misc Ret TD',
       'Field Goal Att', 'Field Goal Made', 'Off XP Kick Att',
       'Off XP Kick Made', 'Off 2XP Att', 'Off 2XP Made', 'Def 2XP Att',
       'Def 2XP Made', 'Safety', 'Punt', 'Punt Yard',
       'Kickoff', 'Kickoff Yard', 'Kickoff Touchback',
       'Kickoff Out-Of-Bounds', 'Kickoff Onside', 'Fumble', 'Fumble Lost',
       'Tackle Solo', 'Tackle Assist', 'Tackle For Loss',
       'Tackle For Loss Yard', 'Sack', 'Sack Yard', 'QB Hurry',
       'Fumble Forced', 'Pass Broken Up', 'Kick/Punt Blocked',
       '1st Down Rush', '1st Down Pass', '1st Down Penalty',
       'Time Of Possession', 'Penalty', 'Penalty Yard', 'Third Down Att',
       'Third Down Conv', 'Fourth Down Att', 'Fourth Down Conv',
       'Red Zone Att', 'Red Zone TD', 'Red Zone Field Goal',
       'First Down Total']

def point_estimator(team):
    clf = linear_model.Lasso()
    train = team[feature_columns]
    model = clf.fit(train, team['Points'])
    return model

def margin_estimator(team):
    clf = linear_model.Lasso()
    train = team[feature_columns]
    model = clf.fit(train, team['Margin'])
    return model

def predict_scores(team, model, iterations, debug=False):
    simulations = pd.DataFrame([kde_random(team[feature], iterations) for feature in feature_columns]).transpose()

    if debug:
        coefficients = pd.DataFrame(model.coef_, feature_columns)
        print coefficients[abs(coefficients[0]) > 0]

    predicted_scores = model.predict(simulations)
    return [max([0, score]) for score in predicted_scores]

k = 10000

bowls = [
    ('Arizona', 'New Mexico'),
    ('Utah', 'BYU'),
    ('Appalachian State', 'Ohio'),
    ('San Jose State', 'Georgia State'),
    ('Louisiana Tech', 'Arkansas State'),
    ('Western Kentucky', 'South Florida'),
    ('Akron', 'Utah State'),
    ('Toledo', 'Temple'),
    ('Boise State', 'Northern Illinois'),
    ('Georgia Southern', 'Bowling Green'),
    ('Western Michigan', 'Middle Tennessee'),
    ('San Diego State', 'Cincinnati'),
    ('Marshall', 'Connecticut'),
    ('Washington State', 'Miami (Florida)'),
    ('Washington', 'Southern Mississippi'),
    ('Duke', 'Indiana'),
    ('Virginia Tech', 'Tulsa'),
    ('Nebraska', 'UCLA'),
    ('Pittsburgh', 'Navy'),
    ('Central Michigan', 'Minnesota'),
    ('Air Force', 'California'),
    ('North Carolina', 'Baylor'),
    ('Nevada', 'Colorado State'),
    ('Texas Tech', 'LSU'),
    ('Memphis', 'Auburn'),
    ('Mississippi State', 'North Carolina State'),
    ('Louisville', 'Texas A&M'),
    ('Wisconsin', 'USC'),
    ('Houston', 'Florida State'),
    ('Clemson', 'Oklahoma'),
    ('Alabama', 'Michigan State'),
    ('Northwestern', 'Tennessee'),
    ('Notre Dame', 'Ohio State'),
    ('Michigan', 'Florida'),
    ('Iowa', 'Stanford'),
    ('Oklahoma State', 'Mississippi'),
    ('Penn State', 'Georgia'),
    ('Kansas State', 'Arkansas'),
    ('Oregon', 'TCU'),
    ('West Virginia', 'Arizona State')
]

winners = [
    ('Arizona', 45, 37),
    ('Utah', 35, 28),
    ('Appalachian State', 31, 29),
    ('San Jose State', 27, 16),
    ('Louisiana Tech', 47, 28),
    ('Western Kentucky', 45, 35),
    ('Akron', 23, 21),
    ('Toledo', 32, 17),
    ('Boise State', 55, 7),
    ('Georgia Southern', 58, 27),
    ('Western Michigan', 45, 31),
    ('San Diego State', 42, 7),
    ('Marshall', 16, 10),
    ('Washington State', 20, 14),
    ('Washington', 44, 31),
    ('Duke', 44, 41),
    ('Virginia Tech', 55, 52),
    ('Nebraska', 37, 29),
    ('Navy', 44, 28),
    ('Minnesota', 21, 14),
    ('California', 55, 36),
    ('Baylor', 49, 38),
    ('Nevada', 28, 23),
    ('LSU', 56, 27)
]

correct = 0

for i, bowl in enumerate(bowls):
    winner = bowl[0]
    team1 = games[games.Name == bowl[0]]
    team2 = games[games.Name == bowl[1]]

    outcome = pd.DataFrame([predict_scores(team1, point_estimator(team1), k), predict_scores(team2, point_estimator(team2), k)]).transpose()
    
    team1_probability = len(outcome[outcome[0] > outcome[1]]) / k
    team2_probability = len(outcome[outcome[1] > outcome[0]]) / k
    spreads = (outcome[0] - outcome[1]).quantile([x / 100 for x in range(0, 100, 10)])

    if team1_probability > team2_probability:
        spreads = (outcome[1] - outcome[0]).quantile([x / 100 for x in range(0, 100, 10)])
        print ', '.join([str(x) for x in [bowl[0], team1_probability, bowl[1], team2_probability]])
    else:
        winner = bowl[1]
        print ', '.join([str(x) for x in [bowl[1], team2_probability, bowl[0], team1_probability]])

    print spreads.iloc[5]
    
    if i < len(winners) and winner == winners[i][0]:
        correct += 1

print 'Accuracy:', correct / len(winners)

Arizona, 0.5889, New Mexico, 0.411
-5.03234456701
Utah, 0.518, BYU, 0.482
-1.2208513588
Appalachian State, 0.7168, Ohio, 0.2831
-13.7828220229
San Jose State, 0.5509, Georgia State, 0.4491
-1.69516445077
Louisiana Tech, 0.5116, Arkansas State, 0.4879
-0.988603515798
Western Kentucky, 0.6749, South Florida, 0.3251
-9.21396138608
Utah State, 0.6218, Akron, 0.3782
-5.86002686933
Toledo, 0.6649, Temple, 0.3351
-6.2352883198
Boise State, 0.5465, Northern Illinois, 0.4535
-3.16863494508
Bowling Green, 0.758, Georgia Southern, 0.242
-9.34113406384
Western Michigan, 0.5213, Middle Tennessee, 0.4758
-1.7379816171
Cincinnati, 0.509, San Diego State, 0.491
-0.46692297147
Marshall, 0.6704, Connecticut, 0.3266
-9.18643132285
Washington State, 0.601, Miami (Florida), 0.399
-5.14984781656
Southern Mississippi, 0.6775, Washington, 0.3213
-13.2893061507
Indiana, 0.5711, Duke, 0.4289
-2.77713608392
Tulsa, 0.6584, Virginia Tech, 0.3416
-7.14903836646
UCLA, 0.5689, Nebraska, 0.431
-4.39322243934
Navy, 0.8